In [1]:
project_id = 'eb3c0e14-6751-4329-b59f-364511a6184e'

cv_key = '99f12a82663e4e4388be35bfd8a0d373' # à réccuperer à partir de Azure RG Prediction Ressource (et non à partir de customVision.ai)

cv_endpoint = 'https://customvisiontpazure.cognitiveservices.azure.com/' # à réccuperer à partir de Azure RG Prediction Ressource (et non à partir de customVision.ai)

model_name = 'dogs-classification' # this must match the model name you set when publishing your model iteration (it's case-sensitive)!
print(f'Ready to predict using model {model_name} in project {project_id}')

Ready to predict using model dogs-classification in project eb3c0e14-6751-4329-b59f-364511a6184e


In [2]:
!pip install azure-cognitiveservices-vision-customvision

In [3]:
!pip install tensorflow

  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azure-core 1.30.0 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
pydantic 2.4.2 requires typing-extensions>=4.6.1, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.10.1 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.


In [4]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
import matplotlib.pyplot as plt
from PIL import Image

In [5]:
# Create an instance of the prediction service
credentials = ApiKeyCredentials(in_headers={"Prediction-key": cv_key})
custom_vision_client = CustomVisionPredictionClient(endpoint=cv_endpoint, credentials=credentials)

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(
        '../images',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

Found 17760 images belonging to 120 classes.


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
datagen = ImageDataGenerator(rescale=1./255)
test_generator = datagen.flow_from_directory(
        '../images',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

Found 17760 images belonging to 120 classes.


In [8]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import CognitiveServicesCredentials
import os
from msrest.authentication import ApiKeyCredentials
from time import sleep
 
# Authentifier le client
# computervision_client = ComputerVisionClient(cv_endpoint, CognitiveServicesCredentials(cv_key))

credentials = ApiKeyCredentials(in_headers={"Training-key": cv_key})
trainer = CustomVisionTrainingClient(cv_endpoint, credentials)
# prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
# predictor = CustomVisionPredictionClient(cv_endpoint, prediction_credentials)

 

## Création du projet


In [9]:
project_name = 'dogs_classification'
project = trainer.create_project(project_name)
project

In [10]:
project.id

'8c465afe-dbdf-4c7c-9250-ba76412b2baa'

In [11]:
image_path = '../images'

folders = os.listdir(image_path)
label = []

for folder in folders:
    # sleep(1)
    trainer.create_tag(project.id, folder) 
    label.append(folder)
    # commenté pour pas recréer les tags


In [12]:
# image_list = []

# for tag in label:
#     categoryName = tag.split('-')
#     categoryName = '_'.join(categoryName).capitalize()

#     for file_name in os.listdir("../images/" + categoryName):
#         file_path = "../images/" + categoryName + "/" + file_name
        
#         with open((file_path), "rb") as image_contents:
#             image_list = []
#             image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[tag]))
#             print(image_list)
#             upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=image_list))
#             if not upload_result.is_batch_successful:
#                 print("Image batch upload failed.")
#                 for image in upload_result.images:
#                     print("Image status: ", image.status)
#                 exit(-1)




In [13]:
project_id = "75b7b86a-4f70-4c9b-b3bc-7bfc40419b29"

## Entrainer l'ia

In [14]:
print ("Training...")
iteration = trainer.train_project(project_id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project_id, iteration.id)
    print ("Training status: " + iteration.status)
    print ("Waiting 10 seconds...")
    sleep(10)

Training...


CustomVisionErrorException: Nothing changed since last training

In [15]:
!pip install azure-cognitiveservices-vision-computervision

  Using cached typing_extensions-4.9.0-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.9.0-py3-none-any.whl (32 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.12.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.9.0 which is incompatible.


In [16]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

# Get endpoint and key from environment variables
import os, requests
endpoint = cv_endpoint
key = cv_key

# Set credentials
credentials = CognitiveServicesCredentials(key)

# Create client
client = ComputerVisionClient(endpoint, credentials)

prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": cv_key})
predictor = CustomVisionPredictionClient(cv_endpoint, prediction_credentials)

In [17]:
publish_iteration_name = "classifyModel"

for tag in label:
    categoryName = tag.split('-')
    categoryName = '_'.join(categoryName).capitalize()

    for file_name in os.listdir("../images/" + categoryName):
        file_path = "../images/" + categoryName + "/" + file_name
        
        image_analysis = client.analyze_image(file_path,visual_features=[VisualFeatureTypes.tags])
        for tag in image_analysis.tags:
            print(tag)


ComputerVisionErrorResponseException: (401) The Analyze Image Operation under Computer Vision API (v3.2) is not supported with the current subscription key and pricing tier Custom_Vision.Training.S0.

## Export du model depuis azure puis importation du model sur le projet python

In [23]:
import tensorflow as tf

# Obtenir le chemin absolu du répertoire actuel
current_directory = os.getcwd()

# Construire le chemin absolu vers le modèle
model_path = os.path.join(current_directory, '../model/model.pb')

# Charger le modèle TensorFlow
with tf.io.gfile.GFile(model_path, 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
    with tf.compat.v1.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')

images_folder = '../images/Affenpinscher/'
image_files = [os.path.join(images_folder, f) for f in os.listdir(images_folder) if os.path.isfile(os.path.join(images_folder, f))]

# Faire des prédictions sur chaque image
predictions = []
with tf.compat.v1.Session(graph=graph) as sess:
    input_op = graph.get_tensor_by_name('input_tensor_name:0')  # Assurez-vous de remplacer 'input_tensor_name' par le nom de votre opération d'entrée
    output_op = graph.get_tensor_by_name('output_tensor_name:0')  # Assurez-vous de remplacer 'output_tensor_name' par le nom de votre opération de sortie

    for image_file in image_files:
        # Charger l'image
        image = cv2.imread(image_file)
        processed_image = preprocess_image(image)

        # Faire une prédiction
        prediction = sess.run(output_op, feed_dict={input_op: processed_image})
        predictions.append(prediction)